In [3]:
# General packages

import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import time
import json
import re


In [ ]:
# Selenium

from selenium.webdriver.edge import service
from selenium import webdriver
from selenium.webdriver.common.by import By
from collections import Counter
from nltk.corpus import stopwords
from selenium.webdriver.common.action_chains import ActionChains

In [34]:
# Load scraped dataset
pages = pd.read_csv("data/full_pages.csv", ";", index_col = "Unnamed: 0")

/home/jane/miniconda2/envs/briq/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (19,20,21,22,23,24,25,28,29) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Collect penetration dates from the forum

In [ ]:
#Load data:

pages_pd = pd.read_csv('scraped_product_pages_extracted.csv', sep=";", index_col = "Unnamed: 0") 

For the ones with a github page:

In [ ]:
pages_penetrated = pages[(pages["nonzero_price"]) & (pages["open_issues"].isnull()==False)]

In [ ]:
pages_penetrated["search_query"]=pages_penetrated["githublink"].str.split('/').str[4]

In [ ]:
pages_penetrated_dict = pages_penetrated[["url", "search_query", "main_developer_username", "launch_date"]].to_dict(orient="records")

In [ ]:
#setting up selenium to function
edgeOption = webdriver.EdgeOptions()
edgeOption.use_chromium = True
edgeOption.add_argument("start-maximized")
edgeOption.binary_location = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe"
s=service.Service(r'C:\Users\Yana Gerina\Documents\msedgedriver.exe')
driver = webdriver.Edge(service=s, options=edgeOption)

In [ ]:
for page in pages_penetrated_dict:
    driver.get("https://forum.xda-developers.com" + page["url"])
    time.sleep(3)
    #Collect the number of open and closed issues if it's available:
    for button in driver.find_elements(by=By.CLASS_NAME, value = "button-text"):
        if button.text == "Search This thread":
            button.click()
            time.sleep(2)
    input_row = driver.switch_to.active_element
    input_row.send_keys("pro |purchase|donation")
    time.sleep(2)
    driver.find_element(by=By.CLASS_NAME, value = "quickSearch-ResultCloser").click()
    time.sleep(1)
    input_row.send_keys(Keys.TAB + Keys.TAB)
    time.sleep(2)
    member_input_row = driver.switch_to.active_element
    member_input_row.send_keys(page["main_developer_username"])
    time.sleep(2)
    member_input_row.send_keys(Keys.ENTER + Keys.ENTER)
    time.sleep(2)
    
    results  = []
    for result in driver.find_elements(by=By.CLASS_NAME, value = "contentRow-main"):
        title = result.find_element(by=By.CLASS_NAME, value = "contentRow-title")
        if re.search("Post", title.text) is not None:
            datestamp = result.find_element(by=By.CLASS_NAME, value = "u-dt").text
            post_number = [x.text for x in result.find_elements(by=By.TAG_NAME, value = "li") if re.search("Post", x.text) is not None][0].replace(",","")
            post_number = int(re.search(r"[\d,]+", post_number).group(0))
            mention = {"title": title.text, "datestamp": datestamp, "post_number": post_number}
            results.append(mention)

        if results:
            earliest_post = min(results, key=lambda x:x['post_number'])
            page["penetration_datestamp"] = earliest_post["datestamp"]
        else: 
            page["penetration_datestamp"] = np.nan
driver.quit()   

In [ ]:
with open('pages_github_penetrated_datestamps.json', 'w') as fp:
    json.dump(pages_penetrated_dict, fp)

For all non-zero priced products:

In [ ]:
pages_nonzero_dict = pages[pages["nonzero_price"]][["url", "search_query", "main_developer_username", "launch_date"]].to_dict(orient="records")

In [ ]:
#setting up selenium to function
edgeOption = webdriver.EdgeOptions()
edgeOption.use_chromium = True
edgeOption.add_argument("start-maximized")
edgeOption.binary_location = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe"
s=service.Service(r'C:\Users\Yana Gerina\Documents\msedgedriver.exe')
driver = webdriver.Edge(service=s, options=edgeOption)

In [ ]:
for page in pages_nonzero_dict:
    driver.get("https://forum.xda-developers.com" + page["url"])
    time.sleep(3)
    #Collect the number of open and closed issues if it's available:
    for button in driver.find_elements(by=By.CLASS_NAME, value = "button-text"):
        if button.text == "Search This thread":
            button.click()
            time.sleep(2)
    input_row = driver.switch_to.active_element
    input_row.send_keys("pro |purchase|donation")
    time.sleep(2)
    driver.find_element(by=By.CLASS_NAME, value = "quickSearch-ResultCloser").click()
    time.sleep(1)
    input_row.send_keys(Keys.TAB + Keys.TAB)
    time.sleep(2)
    member_input_row = driver.switch_to.active_element
    member_input_row.send_keys(page["main_developer_username"])
    time.sleep(2)
    member_input_row.send_keys(Keys.ENTER + Keys.ENTER)
    time.sleep(2)
    
    results  = []
    for result in driver.find_elements(by=By.CLASS_NAME, value = "contentRow-main"):
        title = result.find_element(by=By.CLASS_NAME, value = "contentRow-title")
        if re.search("Post", title.text) is not None:
            datestamp = result.find_element(by=By.CLASS_NAME, value = "u-dt").text
            post_number = [x.text for x in result.find_elements(by=By.TAG_NAME, value = "li") if re.search("Post", x.text) is not None][0].replace(",","")
            post_number = int(re.search(r"[\d,]+", post_number).group(0))
            mention = {"title": title.text, "datestamp": datestamp, "post_number": post_number}
            results.append(mention)
        time.sleep(2)
    while True:
        driver.find_elements(by=By.LINK_TEXT, value = "Next") is not None:
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(2)
        driver.find_element(by=By.LINK_TEXT, value = "Next").click()
        time.sleep(2)
        title = result.find_element(by=By.CLASS_NAME, value = "contentRow-title")
        if re.search("Post", title.text) is not None:
            datestamp = result.find_element(by=By.CLASS_NAME, value = "u-dt").text
            post_number = [x.text for x in result.find_elements(by=By.TAG_NAME, value = "li") if re.search("Post", x.text) is not None][0].replace(",","")
            post_number = int(re.search(r"[\d,]+", post_number).group(0))
            mention = {"title": title.text, "datestamp": datestamp, "post_number": post_number}
            results.append(mention)
            try:
                element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'reviews_pagination_link_nav')))
                element.click()
            except TimeoutException:
                break
            
    if results:
        earliest_post = min(results, key=lambda x:x['post_number'])
        page["penetration_datestamp"] = earliest_post["datestamp"]
    else: 
        page["penetration_datestamp"] = np.nan
driver.quit()    


In [ ]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
time.sleep(2)
driver.find_element(by=By.LINK_TEXT, value = "Next").click()


In [ ]:
driver.find_element(by=By.LINK_TEXT, value = "Next").click()

In [ ]:
driver.quit()

# Collect mentions of product in other forums after the date